In [68]:
import nltk             # for stopwords, for stemming
# nltk.download()
import pandas as pd
import string           # dict of punctuations
import re               # regular expression lib, for matching patterns
import csv

In [70]:
# with open("SMSSpamCollection.tsv", 'r', newline= '') as file:
#     rawData = csv.reader(file)
#     data = list(rawData)

# read the raw file
rawData = open('SMSSpamCollection.tsv').read()

# print first 500 characters
rawData[0:500]

"ham\tI've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times.\nspam\tFree entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's\nham\tNah I don't think he goes to usf, he lives around here though\nham\tEven my brother is not like to speak with me. They treat me like aid"

In [71]:
parsedData = rawData.replace('\t', '\n').split('\n')
parsedData[0:5]

['ham',
 "I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times.",
 'spam',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'ham']

In [72]:
# save the lables (ham) that appears on every other roll and do same for the text(body)
labels = parsedData[0::2]
text = parsedData[1::2]

print(labels[0:10:2])
print(text[0:5])

['ham', 'ham', 'ham', 'spam', 'ham']
["I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times.", "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", "Nah I don't think he goes to usf, he lives around here though", 'Even my brother is not like to speak with me. They treat me like aids patent.', 'I HAVE A DATE ON SUNDAY WITH WILL!!']


In [74]:
fullCorpus = pd.DataFrame({
    'label': labels[:-1],
    'body_list': text
})
fullCorpus

,label,body_list
0,ham,I've been searching for the right words to thank you for this breather. I promise i wont take yo...
1,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...
2,ham,"Nah I don't think he goes to usf, he lives around here though"
3,ham,Even my brother is not like to speak with me. They treat me like aids patent.
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!
...,...,...
5565,spam,This is the 2nd time we have tried 2 contact u. U have won the £750 Pound prize. 2 claim is easy...
5566,ham,Will ü b going to esplanade fr home?
5567,ham,"Pity, * was in mood for that. So...any other suggestions?"
5568,ham,The guy did some bitching but I acted like i'd be interested in buying something else next week ...


In [75]:
# Importing the data the easy way
dataset = pd.read_csv('SMSSpamCollection.tsv', sep='\t', header=None)

# Setting labels for the columns
dataset.columns = ['label', 'body_text']

# View the first 5 rows of the data
dataset.head()

,label,body_text
0,ham,I've been searching for the right words to thank you for this breather. I promise i wont take yo...
1,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...
2,ham,"Nah I don't think he goes to usf, he lives around here though"
3,ham,Even my brother is not like to speak with me. They treat me like aids patent.
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!


In [76]:
# Check the shape of the data
print("Data has {} rows and {} columns".format(
    len(dataset), len(dataset.columns)))

Data has 5568 rows and 2 columns


In [77]:
# Check the ratio of spam to ham
print("Out of {} rows, the number of spam are {} and the number of ham are {}".format(len(
    dataset), len(dataset[dataset['label'] == 'spam']), len(dataset[dataset['label'] == 'ham'])))

Out of 5568 rows, the number of spam are 746 and the number of ham are 4822


In [78]:
# How much missing data do we have
print("There are {} missing labels in our data".format(
    dataset['label'].isnull().sum()))
print("There are {} missing text in our data".format(
    dataset['body_text'].isnull().sum()))

There are 0 missing labels in our data
There are 0 missing text in our data


In [79]:
# Using pandas directly
fullCorpusNew = pd.read_csv('SMSSpamcollection.tsv', sep='\t', header=None)
fullCorpusNew.columns = ['label', 'body_text']
columns = fullCorpusNew.columns
print(fullCorpusNew.columns)

Index(['label', 'body_text'], dtype='object')


In [80]:
fullCorpusNew.head()

,label,body_text
0,ham,I've been searching for the right words to thank you for this breather. I promise i wont take yo...
1,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...
2,ham,"Nah I don't think he goes to usf, he lives around here though"
3,ham,Even my brother is not like to speak with me. They treat me like aids patent.
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!


In [81]:
# what is the shape of the dataset
print('dataset has {} rows and {} colmus'.format(
    len(fullCorpusNew), len(fullCorpusNew.columns)))
print('of the {} dataset, {} are labeled as spam and {} are labeled as ham'.format(len(fullCorpusNew),
                                                                                   len(
    fullCorpusNew[fullCorpusNew['label'] == 'spam']),
    len(fullCorpusNew[fullCorpusNew['label'] == 'ham']))
)

dataset has 5568 rows and 2 colmus
of the 5568 dataset, 746 are labeled as spam and 4822 are labeled as ham


In [82]:
# check for any missing data

print('there are {} missing data in row'.format(
    fullCorpusNew['label'].isnull().sum()))
print('there are {} missing data in row'.format(
    fullCorpusNew['body_text'].isnull().sum()))

there are 0 missing data in row
there are 0 missing data in row


In [83]:
# fullCorpusNew.iloc[3, fullCorpusNew['body_text']]

# split the text by finding any spcial character and make them delimeter for spliting
re.split('\W+', fullCorpusNew['body_text'][0])

['I',
 've',
 'been',
 'searching',
 'for',
 'the',
 'right',
 'words',
 'to',
 'thank',
 'you',
 'for',
 'this',
 'breather',
 'I',
 'promise',
 'i',
 'wont',
 'take',
 'your',
 'help',
 'for',
 'granted',
 'and',
 'will',
 'fulfil',
 'my',
 'promise',
 'You',
 'have',
 'been',
 'wonderful',
 'and',
 'a',
 'blessing',
 'at',
 'all',
 'times',
 '']

In [84]:
# split the text by finding any spcial character or token
re.findall('\w+', fullCorpusNew['body_text'][0])

['I',
 've',
 'been',
 'searching',
 'for',
 'the',
 'right',
 'words',
 'to',
 'thank',
 'you',
 'for',
 'this',
 'breather',
 'I',
 'promise',
 'i',
 'wont',
 'take',
 'your',
 'help',
 'for',
 'granted',
 'and',
 'will',
 'fulfil',
 'my',
 'promise',
 'You',
 'have',
 'been',
 'wonderful',
 'and',
 'a',
 'blessing',
 'at',
 'all',
 'times']

In [85]:
# set the number of data that should be displayed
pd.set_option('display.max_colwidth', 100)

# read data again, since there are no headers, set with column attribute of the df
data = pd.read_csv('SMSSpamCollection.tsv', sep='\t', header=None)
data.columns = ['label', 'body_text']

data.head()

,label,body_text
0,ham,I've been searching for the right words to thank you for this breather. I promise i wont take yo...
1,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...
2,ham,"Nah I don't think he goes to usf, he lives around here though"
3,ham,Even my brother is not like to speak with me. They treat me like aids patent.
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!


In [87]:
# create a list of punctions from the library
punct = string.punctuation

In [88]:
# function to remove punctuations from the dataset, join help keep the individual char together
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in punct])
    return text_nopunct


# create new column without punctuations
data['body_text_clean'] = data['body_text'].apply(lambda x: remove_punct(x))
data.head()

,label,body_text,body_text_clean
0,ham,I've been searching for the right words to thank you for this breather. I promise i wont take yo...,Ive been searching for the right words to thank you for this breather I promise i wont take your...
1,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005 Text FA to 87121 to receive e...
2,ham,"Nah I don't think he goes to usf, he lives around here though",Nah I dont think he goes to usf he lives around here though
3,ham,Even my brother is not like to speak with me. They treat me like aids patent.,Even my brother is not like to speak with me They treat me like aids patent
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,I HAVE A DATE ON SUNDAY WITH WILL


### The below functions could be defined in one function but this was splitted through the learning process

In [89]:
# create func to split words into tokens and store in a list, \W+ regx splits on every non-char words like space or special char
def tokenise(text):
    tokens = re.split("\W+", text)
    return tokens


# apply the function to the desired column in dataframe
data['body_text_tokenised'] = data['body_text_clean'].apply(
    lambda x: tokenise(x.lower()))

In [90]:
data.head()

,label,body_text,body_text_clean,body_text_tokenised
0,ham,I've been searching for the right words to thank you for this breather. I promise i wont take yo...,Ive been searching for the right words to thank you for this breather I promise i wont take your...,"[ive, been, searching, for, the, right, words, to, thank, you, for, this, breather, i, promise, ..."
1,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005 Text FA to 87121 to receive e...,"[free, entry, in, 2, a, wkly, comp, to, win, fa, cup, final, tkts, 21st, may, 2005, text, fa, to..."
2,ham,"Nah I don't think he goes to usf, he lives around here though",Nah I dont think he goes to usf he lives around here though,"[nah, i, dont, think, he, goes, to, usf, he, lives, around, here, though]"
3,ham,Even my brother is not like to speak with me. They treat me like aids patent.,Even my brother is not like to speak with me They treat me like aids patent,"[even, my, brother, is, not, like, to, speak, with, me, they, treat, me, like, aids, patent]"
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,I HAVE A DATE ON SUNDAY WITH WILL,"[i, have, a, date, on, sunday, with, will]"


In [91]:
# remove stopwords. commonly used words that does not contribute much to a sentence
stopwords = nltk.corpus.stopwords.words('english')


def remove_stopwords(tokenised_list):
    text_nostops = [word for word in tokenised_list if word not in stopwords]
    return text_nostops


# cerate a new column from the tokenised words and store in new column
data['body_text_nostops'] = data['body_text_tokenised'].apply(
    lambda x: remove_stopwords(x))
data.head()

,label,body_text,body_text_clean,body_text_tokenised,body_text_nostops
0,ham,I've been searching for the right words to thank you for this breather. I promise i wont take yo...,Ive been searching for the right words to thank you for this breather I promise i wont take your...,"[ive, been, searching, for, the, right, words, to, thank, you, for, this, breather, i, promise, ...","[ive, searching, right, words, thank, breather, promise, wont, take, help, granted, fulfil, prom..."
1,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005 Text FA to 87121 to receive e...,"[free, entry, in, 2, a, wkly, comp, to, win, fa, cup, final, tkts, 21st, may, 2005, text, fa, to...","[free, entry, 2, wkly, comp, win, fa, cup, final, tkts, 21st, may, 2005, text, fa, 87121, receiv..."
2,ham,"Nah I don't think he goes to usf, he lives around here though",Nah I dont think he goes to usf he lives around here though,"[nah, i, dont, think, he, goes, to, usf, he, lives, around, here, though]","[nah, dont, think, goes, usf, lives, around, though]"
3,ham,Even my brother is not like to speak with me. They treat me like aids patent.,Even my brother is not like to speak with me They treat me like aids patent,"[even, my, brother, is, not, like, to, speak, with, me, they, treat, me, like, aids, patent]","[even, brother, like, speak, treat, like, aids, patent]"
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,I HAVE A DATE ON SUNDAY WITH WILL,"[i, have, a, date, on, sunday, with, will]","[date, sunday]"


## The big idea is to help the model to learn better by everything we done, and now let's stem things
#### Stemming

There are different stemming techniques but the most popuplar one porterstemmer was focused on

In [100]:
# parser the stemmer into a variable making an object
ps = nltk.PorterStemmer()

# the concept of stemming
print(ps.stem("grows"))
print(ps.stem("growth"))
print(ps.stem("growing"))
print(ps.stem("grow"))
print(ps.stem("run"))
print(ps.stem("running"))
print(ps.stem("runner"))
print(ps.stem("runneth"))



grow
growth
grow
grow
run
run
runner
runneth


In [114]:
# create new dataframe from the exist with just labels and the tokenised text without stopwords
data_stem = data[['label', 'body_text_nostops']].copy()
data_stem.head()

,label,body_text_nostops
0,ham,"[ive, searching, right, words, thank, breather, promise, wont, take, help, granted, fulfil, prom..."
1,spam,"[free, entry, 2, wkly, comp, win, fa, cup, final, tkts, 21st, may, 2005, text, fa, 87121, receiv..."
2,ham,"[nah, dont, think, goes, usf, lives, around, though]"
3,ham,"[even, brother, like, speak, treat, like, aids, patent]"
4,ham,"[date, sunday]"


In [115]:
# define func to apply stemming to each word in the iterated list
def stemming(tokenised_text):
    text = [ps.stem(word) for word in tokenised_text]
    return text


data_stem['body_text_stemmed'] = data['body_text_nostops'].apply(lambda x: stemming(x))
data_stem.head()

,label,body_text_nostops,body_text_stemmed
0,ham,"[ive, searching, right, words, thank, breather, promise, wont, take, help, granted, fulfil, prom...","[ive, search, right, word, thank, breather, promis, wont, take, help, grant, fulfil, promis, won..."
1,spam,"[free, entry, 2, wkly, comp, win, fa, cup, final, tkts, 21st, may, 2005, text, fa, 87121, receiv...","[free, entri, 2, wkli, comp, win, fa, cup, final, tkt, 21st, may, 2005, text, fa, 87121, receiv,..."
2,ham,"[nah, dont, think, goes, usf, lives, around, though]","[nah, dont, think, goe, usf, live, around, though]"
3,ham,"[even, brother, like, speak, treat, like, aids, patent]","[even, brother, like, speak, treat, like, aid, patent]"
4,ham,"[date, sunday]","[date, sunday]"


In [116]:
# just a random check for missing data again
data_stem.isnull().sum()

label                0
body_text_nostops    0
body_text_stemmed    0
dtype: int64

### Lemmatising
interesting way of reducing words in the corpus, almost similar to stemming, but it uses more informed analysis to create groups of words with similar meaning based on the context.
NB: there are trade-off between accuracy and speed. stem is faster as it chops off the suffix

In [109]:
# load the lemmatiser into an object
wn = nltk.WordNetLemmatizer()

In [112]:
# stemming vs lemmatising
print(ps.stem('meaning'))
print(ps.stem('meanness'))
print(ps.stem('goose'))
print(ps.stem('geese'))
print("\n")
# print(dir(wn))
print(wn.lemmatize("meaning"))
print(wn.lemmatize("meanness"))
print(wn.lemmatize("goose"))
print(wn.lemmatize("geese"))


mean
mean
goos
gees


meaning
meanness
goose
goose


In [121]:
# create function to preform lemmatising over the iterated words inputed
def lemmatising(tokenised_list):
    text  = [wn.lemmatize(word) for word in tokenised_list]
    return text


data['body_text_lemmantised'] = data['body_text_nostops'].apply(lambda x: lemmatising(x))
data.tail(10)

,label,body_text,body_text_clean,body_text_tokenised,body_text_nostops,body_text_lemmantised
5558,ham,Ok lor... Sony ericsson salesman... I ask shuhui then she say quite gd 2 use so i considering...,Ok lor Sony ericsson salesman I ask shuhui then she say quite gd 2 use so i considering,"[ok, lor, sony, ericsson, salesman, i, ask, shuhui, then, she, say, quite, gd, 2, use, so, i, co...","[ok, lor, sony, ericsson, salesman, ask, shuhui, say, quite, gd, 2, use, considering]","[ok, lor, sony, ericsson, salesman, ask, shuhui, say, quite, gd, 2, use, considering]"
5559,ham,Ard 6 like dat lor.,Ard 6 like dat lor,"[ard, 6, like, dat, lor]","[ard, 6, like, dat, lor]","[ard, 6, like, dat, lor]"
5560,ham,Why don't you wait 'til at least wednesday to see if you get your .,Why dont you wait til at least wednesday to see if you get your,"[why, dont, you, wait, til, at, least, wednesday, to, see, if, you, get, your, ]","[dont, wait, til, least, wednesday, see, get, ]","[dont, wait, til, least, wednesday, see, get, ]"
5561,ham,Huh y lei...,Huh y lei,"[huh, y, lei]","[huh, lei]","[huh, lei]"
5562,spam,REMINDER FROM O2: To get 2.50 pounds free call credit and details of great offers pls reply 2 th...,REMINDER FROM O2 To get 250 pounds free call credit and details of great offers pls reply 2 this...,"[reminder, from, o2, to, get, 250, pounds, free, call, credit, and, details, of, great, offers, ...","[reminder, o2, get, 250, pounds, free, call, credit, details, great, offers, pls, reply, 2, text...","[reminder, o2, get, 250, pound, free, call, credit, detail, great, offer, pls, reply, 2, text, v..."
5563,spam,This is the 2nd time we have tried 2 contact u. U have won the £750 Pound prize. 2 claim is easy...,This is the 2nd time we have tried 2 contact u U have won the £750 Pound prize 2 claim is easy c...,"[this, is, the, 2nd, time, we, have, tried, 2, contact, u, u, have, won, the, 750, pound, prize,...","[2nd, time, tried, 2, contact, u, u, 750, pound, prize, 2, claim, easy, call, 087187272008, now1...","[2nd, time, tried, 2, contact, u, u, 750, pound, prize, 2, claim, easy, call, 087187272008, now1..."
5564,ham,Will ü b going to esplanade fr home?,Will ü b going to esplanade fr home,"[will, ü, b, going, to, esplanade, fr, home]","[ü, b, going, esplanade, fr, home]","[ü, b, going, esplanade, fr, home]"
5565,ham,"Pity, * was in mood for that. So...any other suggestions?",Pity was in mood for that Soany other suggestions,"[pity, was, in, mood, for, that, soany, other, suggestions]","[pity, mood, soany, suggestions]","[pity, mood, soany, suggestion]"
5566,ham,The guy did some bitching but I acted like i'd be interested in buying something else next week ...,The guy did some bitching but I acted like id be interested in buying something else next week a...,"[the, guy, did, some, bitching, but, i, acted, like, id, be, interested, in, buying, something, ...","[guy, bitching, acted, like, id, interested, buying, something, else, next, week, gave, us, free]","[guy, bitching, acted, like, id, interested, buying, something, else, next, week, gave, u, free]"
5567,ham,Rofl. Its true to its name,Rofl Its true to its name,"[rofl, its, true, to, its, name]","[rofl, true, name]","[rofl, true, name]"


## Vectorisation

### count vectorization

Taking the tokenised stemmed or lemmatised text row by row or say line by line and counting eah word occurence as they appear all through the document, counting each word per cell.
This words form columns and are called document term matrix.

### 